# FAQ Regeneration Detector - Simplified Checksum-Centric Architecture

## 🎯 Purpose

This notebook detects which content changes require FAQ regeneration using a **simplified checksum-centric approach**:
- **Binary Decision Model**: `requires_faq_regeneration` (0=preserve, 1=regenerate)
- **Checksum-Based Identity**: Checksums are primary identity for FAQ mapping
- **FAQ Impact Tracking**: Shows how many FAQs will be affected by changes
- **Direct Processing**: No adapter layer needed - direct detection result processing

### ✨ Key Architecture Changes (v4 Schema)

**Simplified content_change_log table**:
- Binary decision: `requires_faq_regeneration` BOOLEAN (0/1) instead of complex `change_type` enum
- JSON metadata storage: All optional fields (page, title, domain, etc.) in flexible `metadata` JSON column
- Checksum-centric: `content_checksum` is primary identity (not page numbers)
- Built-in telemetry: Detection metrics included in result (no separate reporter needed)
- Idempotent: UNIQUE constraint on (content_id, detection_run_id) for safe re-runs

**Simplified faq_content_map table**:
- Architectural consistency: Also uses JSON metadata (`current_metadata`, `original_metadata`)
- Page numbers moved to JSON: `json_extract(current_metadata, '$.current_page')`
- Checksum-based invalidation: Single operation per content checksum

**Removed complexity**:
- No `content_diffs` table (diff tracking removed)
- No `FAQToChangesAdapter` (direct processing)
- No complex change classification (content_edit, position_change, etc.)
- No separate impact reporter (use telemetry)

### 📊 Key Features

1. **Simplified Detection Logic**
   - Only 2 categories: `regenerate_faq` vs `no_action_needed`
   - Checksum-based: "Does this checksum exist in file's baseline?"
   - No complex change classification needed
   - Direct answer to business question: "Should we regenerate FAQ?"

2. **JSON Metadata Flexibility**
   - All optional fields stored as JSON (page, title, domain, service, etc.)
   - Easy to add new metadata without schema changes
   - Indexed JSON extraction for common queries
   - SQLite JSON1 extension for querying

3. **FAQ Impact Analysis**
   - Count existing FAQs per checksum
   - Track FAQs to invalidate vs preserve
   - Calculate FAQ generation savings
   - Prioritization with `v_regeneration_queue` VIEW

4. **Actionable Output**
   - Clear list of pages needing FAQ regeneration
   - Reason: new checksum not in baseline
   - Ready for FAQ generation pipeline
   - Comprehensive telemetry metrics

### ⚠️ Prerequisites

- Database must exist: `../databases/faq_update.db`
- Schema v4 must be applied (simplified checksum-centric schema)
- Content changes must be present in `content_repo` table
- Run `setup_tables.ipynb` first if database doesn't exist
- Run `content_edit.ipynb` to create test content changes

### 📖 When to Run

- Before starting FAQ generation process
- Daily batch processing to identify pages needing FAQs
- Testing FAQ regeneration logic
- Calculating FAQ generation workload

### 🏗️ Workflow Steps

1. **Detection** (Step 5): Identify which pages require FAQ regeneration
2. **Logging** (Step 6): Record detection results in `content_change_log`
3. **FAQ Updates** (Step 7): Invalidate FAQs for regeneration, update metadata for preserved
4. **Verification** (Step 8): Query results using simplified schema
5. **Summary**: Review telemetry and next steps

## Step 1: Configuration and Imports

In [1]:
import sys
import sqlite3
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
import logging

# ============================================================================
# 📁 PATH CONFIGURATION
# ============================================================================

# Project root and source directory
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent
SRC_DIR = PROJECT_ROOT / 'src'

# Add src to Python path
sys.path.insert(0, str(SRC_DIR))

# Database paths
DATABASE_DIR = PROJECT_ROOT / 'databases'
DATABASE_NAME = 'faq_update.db'
DB_PATH = DATABASE_DIR / DATABASE_NAME

# ============================================================================
# 📊 DISPLAY CONFIGURATION
# ============================================================================

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 60)

print("=" * 100)
print("🚀 UPDATE DETECTOR - Content Change Detection and Processing")
print("=" * 100)
print(f"📁 Project Root: {PROJECT_ROOT.absolute()}")
print(f"📁 Database: {DB_PATH.absolute()}")
print(f"📁 Source Directory: {SRC_DIR.absolute()}")
print("=" * 100)

🚀 UPDATE DETECTOR - Content Change Detection and Processing
📁 Project Root: c:\Users\praka\projects\FAQ\FAQ_update
📁 Database: c:\Users\praka\projects\FAQ\FAQ_update\databases\faq_update.db
📁 Source Directory: c:\Users\praka\projects\FAQ\FAQ_update\src


In [2]:
# Import utility functions from src/utils
from utils import (
    BASE_TIME,
    format_timestamp,
    get_timestamp,
    calculate_checksum,
    calculate_file_checksum,
    validate_checksum,
    create_markdown_file,
    compress_diff,
    decompress_diff,
    preview_dataframe,
    print_config_banner,
    log_success,
    log_info,
    log_warning,
    log_error
)

# Import pipeline components
from validators.content_validator import ContentValidator
from detectors.faq_regeneration_detector import FAQRegenerationDetector

# Import NEW simplified processing components (checksum-centric architecture)
from processors.change_processor import ChangeLogger  # Replaces ChangeProcessor
from processors.faq_mapper import FAQUpdater  # Replaces FAQMapper

# Import human-readable diff formatter (if needed)
from formatters.diff_formatter import DiffFormatter

# NOTE: Following components are REMOVED/DEPRECATED in simplified architecture:
# - FAQToChangesAdapter: No longer needed (direct detection result processing)
# - ChangeProcessor: Deprecated (use ChangeLogger instead)
# - FAQMapper: Deprecated (use FAQUpdater instead)
# - ContentDiffProcessor: Removed (no more content_diffs table)
# - ImpactReporter: Simplified (use detection telemetry instead)

## ⚠️ IMPORTANT: Module Reloading

**If you've made changes to the source code**, you need to restart the kernel to reload the modules:
- **Jupyter Notebook**: Kernel → Restart Kernel
- **JupyterLab**: Kernel → Restart Kernel
- **VS Code**: Click the restart button in the notebook toolbar

Alternatively, run the cell below to force-reload all modules (less reliable):

In [3]:
# Optional: Force reload modules (use if you can't restart kernel)
# WARNING: This is less reliable than restarting the kernel
import sys
import importlib

# Remove cached modules
modules_to_reload = [
    'processors.change_processor',
    'processors.faq_mapper',
    'detectors.faq_regeneration_detector'
]

for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]
        print(f"Removed {module_name} from cache")

print("\n✅ Module cache cleared. Re-run import cells below.")

Removed processors.change_processor from cache
Removed processors.faq_mapper from cache
Removed detectors.faq_regeneration_detector from cache

✅ Module cache cleared. Re-run import cells below.


In [4]:
# ##  check and conditionally delete all content from content_change_log:

# with sqlite3.connect(DB_PATH) as conn:
#     count = conn.execute("SELECT COUNT(*) FROM content_change_log").fetchone()[0]
#     print(f"Found {count} records in content_change_log")
#     if count > 0:
#         conn.execute("DELETE FROM content_change_log")
#         print(f"✅ Deleted {count} records")
#     else:
#         print("ℹ️  No records to delete")

## Step 2: Verify Database and Initial State

In [5]:
print("\n" + "="*100)
print("DATABASE VERIFICATION")
print("="*100)

# Check database exists
if not DB_PATH.exists():
    log_error(f"Database not found: {DB_PATH}")
    log_error("Please run 'setup_tables.ipynb' first to create the database")
    raise FileNotFoundError(f"Database file not found: {DB_PATH}")

log_success(f"Database found: {DB_PATH.name}")

# Connect and verify tables
with sqlite3.connect(DB_PATH) as conn:
    # Enable foreign keys
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Verify required tables exist (simplified schema - no content_diffs)
    required_tables = [
        'content_repo',
        'content_change_log',
        'faq_content_map',
        'faq_questions',
        'faq_answers'
    ]
    
    for table_name in required_tables:
        result = conn.execute(
            "SELECT name FROM sqlite_master WHERE type='table' AND name=?",
            (table_name,)
        ).fetchone()
        
        if result:
            count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
            log_success(f"Table '{table_name}' exists ({count} records)")
        else:
            log_error(f"Required table '{table_name}' not found!")
            raise ValueError(f"Database missing required table: {table_name}")
    
    # Show current state
    print("\n📊 Current Database State:")
    
    # Content repo summary
    content_stats = pd.read_sql_query("""
        SELECT 
            COUNT(*) as total_pages,
            COUNT(DISTINCT raw_file_nme) as unique_files,
            COUNT(DISTINCT content_checksum) as unique_checksums,
            SUM(CASE WHEN file_status = 'Active' THEN 1 ELSE 0 END) as active_pages,
            MAX(last_modified_dt) as latest_modification
        FROM content_repo
    """, conn)
    
    print("\n  Content Repository:")
    print(f"    Total Pages: {content_stats['total_pages'].iloc[0]}")
    print(f"    Unique Files: {content_stats['unique_files'].iloc[0]}")
    print(f"    Unique Checksums: {content_stats['unique_checksums'].iloc[0]}")
    print(f"    Active Pages: {content_stats['active_pages'].iloc[0]}")
    print(f"    Latest Modification: {content_stats['latest_modification'].iloc[0]}")
    
    # Tracking tables summary
    change_count = conn.execute("SELECT COUNT(*) FROM content_change_log").fetchone()[0]
    faq_map_count = conn.execute("SELECT COUNT(*) FROM faq_content_map").fetchone()[0]
    
    print("\n  Tracking Tables:")
    print(f"    Content Change Log: {change_count} records")
    print(f"    FAQ Content Map: {faq_map_count} mappings")

print("\n" + "="*100)


DATABASE VERIFICATION
✅ Database found: faq_update.db
✅ Table 'content_repo' exists (6 records)
✅ Table 'content_change_log' exists (0 records)
✅ Table 'faq_content_map' exists (10 records)
✅ Table 'faq_questions' exists (10 records)
✅ Table 'faq_answers' exists (10 records)

📊 Current Database State:

  Content Repository:
    Total Pages: 6
    Unique Files: 1
    Unique Checksums: 5
    Active Pages: 6
    Latest Modification: 2025-10-20T03:47:22.000Z

  Tracking Tables:
    Content Change Log: 0 records
    FAQ Content Map: 10 mappings



In [6]:
# with sqlite3.connect(DB_PATH) as conn:
#     # Query all records from content_repo
#     all_records_df = pd.read_sql_query("""
#         SELECT *
#         FROM content_repo
#         ORDER BY ud_source_file_id
#     """, conn)

# all_records_df

In [7]:
with sqlite3.connect(DB_PATH) as conn:
    # Query all records from content_repo
    all_records_df = pd.read_sql_query("""
        SELECT *
        FROM content_change_log
    """, conn)

all_records_df

Empty DataFrame
Columns: [change_id, content_id, content_checksum, file_name, requires_faq_regeneration, metadata, detected_at, source_modified_at, existing_faq_count, detection_run_id, since_date]
Index: []

In [8]:
with sqlite3.connect(DB_PATH) as conn:
    # Query all records from faq_content_map
    # NOTE: Simplified schema - page numbers now in JSON metadata (current_metadata, original_metadata)
    all_records_df = pd.read_sql_query("""
        SELECT
            map_id,
            question_id,
            answer_id,
            content_checksum,
            current_content_id,
            json_extract(current_metadata, '$.current_page') as current_page,
            current_file_name,
            original_content_id,
            json_extract(original_metadata, '$.original_page') as original_page,
            created_for_version,
            is_valid,
            valid_from,
            valid_until,
            invalidation_reason,
            invalidated_by_change_id,
            replaced_by_map_id,
            confidence_score,
            generation_method,
            created_at,
            last_validated_at
        FROM faq_content_map
    """, conn)

all_records_df

map_id  question_id  answer_id  \
0       1         1001       1001   
1       2         1002       1002   
2       3         1003       1003   
3       4         1004       1004   
4       5         1005       1005   
5       6         1006       1006   
6       7         1007       1007   
7       8         1008       1008   
8       9         1009       1009   
9      10         1010       1010   

                                              content_checksum  \
0  620c75e45bb1c3a1225a411c9821805c866c60fe8e6a61c8586ab3ac...   
1  620c75e45bb1c3a1225a411c9821805c866c60fe8e6a61c8586ab3ac...   
2  620c75e45bb1c3a1225a411c9821805c866c60fe8e6a61c8586ab3ac...   
3  b04ed99ce1bf45e408ed7a519c4d4d65554fcccaa7e3fbab8ba37639...   
4  b04ed99ce1bf45e408ed7a519c4d4d65554fcccaa7e3fbab8ba37639...   
5  b04ed99ce1bf45e408ed7a519c4d4d65554fcccaa7e3fbab8ba37639...   
6  b04ed99ce1bf45e408ed7a519c4d4d65554fcccaa7e3fbab8ba37639...   
7  1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca...   
8  1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca...   
9  1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca...   

   current_content_id  current_page          current_file_name  \
0                1001             1  Employee_Leave_Policy.pdf   
1                1001             1  Employee_Leave_Policy.pdf   
2                1001             1  Employee_Leave_Policy.pdf   
3                1002             2  Employee_Leave_Policy.pdf   
4                1002             2  Employee_Leave_Policy.pdf   
5                1002             2  Employee_Leave_Policy.pdf   
6                1002             2  Employee_Leave_Policy.pdf   
7                1003             3  Employee_Leave_Policy.pdf   
8                1003             3  Employee_Leave_Policy.pdf   
9                1003             3  Employee_Leave_Policy.pdf   

   original_content_id  original_page  created_for_version  is_valid  \
0                 1001              1                    1         1   
1                 1001              1                    1         1   
2                 1001              1                    1         1   
3                 1002              2                    1         1   
4                 1002              2                    1         1   
5                 1002              2                    1         1   
6                 1002              2                    1         1   
7                 1003              3                    1         1   
8                 1003              3                    1         1   
9                 1003              3                    1         1   

            valid_from valid_until invalidation_reason  \
0  2025-10-20 03:46:46        None                None   
1  2025-10-20 03:46:46        None                None   
2  2025-10-20 03:46:46        None                None   
3  2025-10-20 03:46:46        None                None   
4  2025-10-20 03:46:46        None                None   
5  2025-10-20 03:46:46        None                None   
6  2025-10-20 03:46:46        None                None   
7  2025-10-20 03:46:46        None                None   
8  2025-10-20 03:46:46        None                None   
9  2025-10-20 03:46:46        None                None   

  invalidated_by_change_id replaced_by_map_id confidence_score  \
0                     None               None             None   
1                     None               None             None   
2                     None               None             None   
3                     None               None             None   
4                     None               None             None   
5                     None               None             None   
6                     None               None             None   
7                     None               None             None   
8                     None               None             None   
9                     None               None    

## Step 3: Configure Detection Parameters

In [9]:
print("\n" + "="*100)
print("DETECTION PARAMETERS")
print("="*100)

# ============================================================================
# CONFIGURE THESE PARAMETERS
# ============================================================================

# Option 1: Detect changes since a specific date
SINCE_DATE = '2025-10-17T17:01:51.100Z'  # Adjust as needed

# Option 2: Detect changes since last run (uncomment to use)
# with sqlite3.connect(DB_PATH) as conn:
#     last_run = conn.execute("""
#         SELECT MAX(detected_at) FROM content_change_log
#     """).fetchone()[0]
#     SINCE_DATE = last_run if last_run else '2025-01-01T00:00:00Z'

# Option 3: Process all content (first run)
# SINCE_DATE = '1970-01-01T00:00:00Z'

print(f"\n📅 Detection Period:")
print(f"  Since Date: {SINCE_DATE}")

# Show what will be processed
with sqlite3.connect(DB_PATH) as conn:
    pending_content = pd.read_sql_query("""
        SELECT 
            COUNT(*) as count,
            COUNT(DISTINCT raw_file_nme) as files
        FROM content_repo
        WHERE last_modified_dt > ?
    """, conn, params=[SINCE_DATE])
    
    print(f"\n📋 Content to Process:")
    print(f"  Pages Modified Since {SINCE_DATE}: {pending_content['count'].iloc[0]}")
    print(f"  Files Affected: {pending_content['files'].iloc[0]}")

print("\n" + "="*100)


DETECTION PARAMETERS

📅 Detection Period:
  Since Date: 2025-10-17T17:01:51.100Z

📋 Content to Process:
  Pages Modified Since 2025-10-17T17:01:51.100Z: 3
  Files Affected: 1



## Step 4: Initialize Components

In [10]:
print("\n" + "="*100)
print("INITIALIZING COMPONENTS")
print("="*100)

# Convert Path to string for module compatibility
db_path_str = str(DB_PATH.absolute())

# Initialize components
try:
    # FAQ Detection
    faq_detector = FAQRegenerationDetector(db_path_str, db_path_str)
    log_success("FAQRegenerationDetector initialized")
    
    # NEW: Simplified processing components (checksum-centric architecture)
    change_logger = ChangeLogger(db_path_str)
    log_success("ChangeLogger initialized (replaces ChangeProcessor)")
    
    faq_updater = FAQUpdater(db_path_str)
    log_success("FAQUpdater initialized (replaces FAQMapper)")
    
    # Optional: Diff formatter (for human-readable diff display if needed)
    diff_formatter = DiffFormatter(db_path_str, db_path_str)
    log_success("DiffFormatter initialized")
    
except Exception as e:
    log_error(f"Failed to initialize components: {e}")
    raise

print("\n" + "="*100)
log_success("All components initialized successfully")
print("\nℹ️  Note: Using simplified checksum-centric architecture")
print("   - ChangeLogger replaces ChangeProcessor (binary decision model)")
print("   - FAQUpdater replaces FAQMapper (checksum-based invalidation)")
print("   - No adapter needed (direct detection result processing)")


INITIALIZING COMPONENTS
✅ FAQRegenerationDetector initialized
✅ ChangeLogger initialized (replaces ChangeProcessor)
✅ FAQUpdater initialized (replaces FAQMapper)
✅ DiffFormatter initialized

✅ All components initialized successfully

ℹ️  Note: Using simplified checksum-centric architecture
   - ChangeLogger replaces ChangeProcessor (binary decision model)
   - FAQUpdater replaces FAQMapper (checksum-based invalidation)
   - No adapter needed (direct detection result processing)


## Step 5: FAQ Regeneration Detection

**Simplified FAQ-focused detection (Checksum-Centric Model)**

This step uses the FAQRegenerationDetector to answer the key business question:
"Which pages require FAQ regeneration?"

**Logic (Checksum-Centric)**:
- Compare current checksums against baseline (historical checksums for this file)
- If checksum NOT in baseline → New/modified content → **REGENERATE FAQ**
- If checksum IN baseline → Content reused → **NO regeneration needed**

**Benefits**:
- Page reordering doesn't trigger FAQ regeneration
- Content duplication is detected (reuses existing FAQs)
- Only truly new/modified content triggers regeneration
- Tracks FAQ count impact (invalidated vs preserved)

**Important Notes**:
- `page_number` is metadata only (for human reference)
- Detection is purely based on checksum existence in file's history
- Simplified model doesn't track per-content_id history (no old_checksum)
- All changes classified as either "new_content" or "content_reused"

In [11]:
faq_regen_result = faq_detector.detect(SINCE_DATE)
faq_regen_result

{'regenerate_faq': [{'content_id': 1004,
   'file_name': 'Employee_Leave_Policy.pdf',
   'checksum': 'ce9d64d784d0d2618919b6d3a49601785095b21f968c06f71b92a8e1b05be3ef',
   'last_modified_dt': '2025-10-17T17:01:58.000Z',
   'metadata': {'page': 2,
    'domain': 'HR',
    'service': 'Policy Management',
    'orgoid': 'ORG001',
    'associateoid': 'ASSOC001',
    'raw_file_type': '.pdf',
    'raw_file_version_nbr': 2,
    'source_url_txt': 'https://intranet.example.com/policies/leave',
    'parent_location_txt': '/policies',
    'raw_file_path': '/test/Employee_Leave_Policy.pdf',
    'extracted_markdown_file_path': 'c:\\Users\\praka\\projects\\FAQ\\FAQ_update\\test_data_version_tracking\\markdown_files\\leave_policy\\page2_v2.md',
    'extracted_layout_file_path': 'layout_p2_v2.txt',
    'title_nme': 'Employee Leave Policy - Sick Leave',
    'breadcrumb_txt': 'Policies > HR > Leave Management',
    'content_tags_txt': 'leave;sick;medical;absence',
    'version_nbr': 2,
    'file_status': 

In [12]:
print("\n" + "="*100)
print("STEP 1: FAQ REGENERATION ANALYSIS")
print("="*100)

start_time = datetime.now()
log_info(f"Starting FAQ regeneration analysis at {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

try:
    # Detect which pages need FAQ regeneration (includes comprehensive telemetry)
    faq_regen_result = faq_detector.detect(SINCE_DATE)
    
    # Extract telemetry for reporting
    telemetry = faq_regen_result['telemetry']
    
    # Display summary from telemetry
    print(f"\n📊 FAQ Regeneration Analysis (completed in {telemetry['detection_duration_seconds']:.2f}s):")
    print(f"  Pages Requiring FAQ Regeneration: {telemetry['pages_requiring_regeneration']}")
    print(f"  Pages NOT Requiring Regeneration: {telemetry['pages_not_requiring_regeneration']}")
    print(f"  FAQs to Invalidate: {telemetry['faqs_to_invalidate']}")
    print(f"  FAQs Preserved: {telemetry['faqs_preserved']}")
    
    # Display operational metrics
    print(f"\n🔧 Operational Metrics:")
    print(f"  Files Processed: {telemetry['files_processed']}")
    print(f"  Total Pages Analyzed: {telemetry['total_pages_analyzed']}")
    print(f"  Pages with Missing Checksums: {telemetry['pages_with_missing_checksums']}")
    print(f"  Pages with No Baseline: {telemetry['pages_with_no_baseline']}")
    print(f"  Total Baseline Checksums Loaded: {telemetry['total_baseline_checksums_loaded']}")
    
    # Display timing details
    print(f"\n⏱️  Timing:")
    print(f"  Started At: {telemetry['detection_started_at']}")
    print(f"  Completed At: {telemetry['detection_completed_at']}")
    print(f"  Duration: {telemetry['detection_duration_seconds']:.3f}s")
    
    # Show warnings if any
    if telemetry['warnings']:
        print(f"\n⚠️  Warnings ({len(telemetry['warnings'])}):")
        for warning in telemetry['warnings'][:10]:  # Show first 10
            print(f"    - {warning}")
        if len(telemetry['warnings']) > 10:
            print(f"    ... and {len(telemetry['warnings']) - 10} more warnings")
    
    # Show per-file breakdown
    if telemetry['files_detail']:
        print(f"\n📁 Per-File Breakdown:")
        for file_detail in telemetry['files_detail']:
            print(f"\n  File: {file_detail['file_name']}")
            print(f"    Baseline Checksums: {file_detail['baseline_checksums_count']}")
            print(f"    Pages Analyzed: {file_detail['pages_analyzed']}")
            print(f"    Pages Requiring Regeneration: {file_detail['pages_requiring_regeneration']}")
            print(f"    Pages NOT Requiring Regeneration: {file_detail['pages_not_requiring_regeneration']}")
            print(f"    Pages with Missing Checksums: {file_detail['pages_with_missing_checksums']}")
            print(f"    FAQs to Invalidate: {file_detail['faqs_to_invalidate']}")
            print(f"    FAQs Preserved: {file_detail['faqs_preserved']}")
    
    # Show pages requiring regeneration
    if faq_regen_result['regenerate_faq']:
        print("\n📝 Pages Requiring FAQ Regeneration:")
        regen_df = pd.DataFrame(faq_regen_result['regenerate_faq'])
        
        # Extract page from metadata (may be None for single-page docs)
        regen_df['page'] = regen_df['metadata'].apply(lambda m: m.get('page') if isinstance(m, dict) else None)
        
        # Display core + metadata fields
        display_cols = ['file_name', 'page', 'checksum', 'existing_faq_count']
        preview_dataframe(
            regen_df[display_cols],
            "FAQ Regeneration Required",
            max_rows=10
        )
    
    # Show pages NOT requiring regeneration (content reused)
    if faq_regen_result['no_action_needed']:
        print("\n✓ Pages NOT Requiring FAQ Regeneration (Content Reused from Baseline):")
        no_action_df = pd.DataFrame(faq_regen_result['no_action_needed'])
        
        # Extract page from metadata (may be None for single-page docs)
        no_action_df['page'] = no_action_df['metadata'].apply(lambda m: m.get('page') if isinstance(m, dict) else None)
        
        preview_dataframe(
            no_action_df[['file_name', 'page', 'checksum', 'existing_faq_count']],
            "FAQ Regeneration NOT Required",
            max_rows=10
        )
    
    # Savings calculation
    saved_faqs = telemetry['faqs_preserved']
    if saved_faqs > 0:
        print(f"\n💡 FAQ Generation Savings: {saved_faqs} FAQs preserved (no regeneration needed)")

    log_success(f"FAQ regeneration analysis completed")
    
except Exception as e:
    log_error(f"FAQ regeneration analysis failed: {e}")
    raise

print("\n" + "="*100)


STEP 1: FAQ REGENERATION ANALYSIS
ℹ️  Starting FAQ regeneration analysis at 2025-10-19 23:28:10

📊 FAQ Regeneration Analysis (completed in 0.01s):
  Pages Requiring FAQ Regeneration: 2
  Pages NOT Requiring Regeneration: 1
  FAQs to Invalidate: 0
  FAQs Preserved: 0

🔧 Operational Metrics:
  Files Processed: 1
  Total Pages Analyzed: 3
  Pages with Missing Checksums: 0
  Pages with No Baseline: 0
  Total Baseline Checksums Loaded: 3

⏱️  Timing:
  Started At: 2025-10-19T23:28:10.771386
  Completed At: 2025-10-19T23:28:10.782299
  Duration: 0.011s

📁 Per-File Breakdown:

  File: Employee_Leave_Policy.pdf
    Baseline Checksums: 3
    Pages Analyzed: 3
    Pages Requiring Regeneration: 2
    Pages NOT Requiring Regeneration: 1
    Pages with Missing Checksums: 0
    FAQs to Invalidate: 0
    FAQs Preserved: 0

📝 Pages Requiring FAQ Regeneration:

📄 FAQ Regeneration Required Preview (first 2 of 2 rows):
--------------------------------------------------------------------------------
    

## Step 5.5: Clear Existing Data (Optional - For Testing)

**Clear existing data in tracking tables before logging new detection results**

This is useful when re-running the notebook to avoid duplicate entries.

In [13]:
# Optional: Clear existing data in tracking tables
# Uncomment the lines below if you want to start fresh

print("\n" + "="*100)
print("CLEARING EXISTING DATA (Optional)")
print("="*100)

with sqlite3.connect(DB_PATH) as conn:
    # Check current counts before deletion
    ccl_count_before = conn.execute("SELECT COUNT(*) FROM content_change_log").fetchone()[0]
    fcm_count_before = conn.execute("SELECT COUNT(*) FROM faq_content_map").fetchone()[0]
    
    print(f"\nCurrent record counts:")
    print(f"  content_change_log: {ccl_count_before} records")
    print(f"  faq_content_map: {fcm_count_before} records")
    
    # DELETE existing data (uncomment to execute)
    # DANGER: This will delete all data in these tables!
    
    # conn.execute("DELETE FROM content_change_log")
    # conn.execute("DELETE FROM faq_content_map")
    # conn.commit()
    
    # log_warning("Data cleared from content_change_log and faq_content_map")
    
    # Uncommented version (safe - just shows what would be deleted)
    print(f"\n⚠️  To clear data, uncomment the DELETE statements in this cell")
    print(f"   This will delete {ccl_count_before + fcm_count_before} total records")

print("=" * 100)


CLEARING EXISTING DATA (Optional)

Current record counts:
  content_change_log: 3 records
  faq_content_map: 0 records

⚠️  To clear data, uncomment the DELETE statements in this cell
   This will delete 3 total records


## Step 6: Log Detection Results (Simplified Checksum-Centric Architecture)

**Record detection results in content_change_log using ChangeLogger**

This step persists the FAQ regeneration detection results to the `content_change_log` table using the simplified checksum-centric architecture:

**Key Changes**:
- No adapter needed - direct processing of detection results
- Binary decision model: `requires_faq_regeneration` (0 or 1)
- JSON metadata storage for optional fields (page, title, domain, etc.)
- Checksum is primary identity for FAQ mapping
- Idempotent writes (safe to re-run detection)

In [14]:
print("\n" + "="*100)
print("STEP 2: LOG DETECTION RESULTS TO content_change_log")
print("="*100)

start_time = datetime.now()
log_info(f"Starting change logging at {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

try:
    # Log detection results using ChangeLogger (checksum-centric architecture)
    # No adapter needed - direct processing of detection result
    log_results = change_logger.log_detection_results(
        faq_regen_result,
        detection_run_id=f"run_{start_time.strftime('%Y%m%d_%H%M%S')}"
    )
    
    log_time = (datetime.now() - start_time).total_seconds()
    
    print(f"\n📊 Change Logging Results (completed in {log_time:.2f}s):")
    print(f"  Records Inserted: {log_results['records_inserted']}")
    print(f"  Records Requiring Regeneration: {log_results['records_requiring_regen']}")
    print(f"  Records Preserving FAQs: {log_results['records_preserving']}")
    print(f"  Duplicates Skipped: {log_results['duplicates_skipped']}")
    print(f"  Errors: {len(log_results['errors'])}")
    
    if log_results['errors']:
        log_warning(f"Encountered {len(log_results['errors'])} errors during logging")
        for error in log_results['errors'][:5]:
            print(f"    - {error}")
    
    # Show sample of logged records with JSON metadata extraction
    print("\n📋 Sample Logged Records (with JSON metadata extraction):")
    with sqlite3.connect(DB_PATH) as conn:
        sample_df = pd.read_sql_query("""
            SELECT 
                change_id,
                content_id,
                file_name,
                content_checksum,
                requires_faq_regeneration,
                json_extract(metadata, '$.page') as page,
                json_extract(metadata, '$.title_nme') as title,
                json_extract(metadata, '$.domain') as domain,
                existing_faq_count,
                detected_at
            FROM content_change_log
            ORDER BY detected_at DESC
            LIMIT 10
        """, conn)
        
        preview_dataframe(sample_df, "Recent Change Log Records", max_rows=10)
    
    log_success(f"Detection results logged to content_change_log")
    
except Exception as e:
    log_error(f"Change logging failed: {e}")
    raise

print("\n" + "="*100)


STEP 2: LOG DETECTION RESULTS TO content_change_log
ℹ️  Starting change logging at 2025-10-19 23:28:10

📊 Change Logging Results (completed in 0.01s):
  Records Inserted: 3
  Records Requiring Regeneration: 2
  Records Preserving FAQs: 1
  Duplicates Skipped: 0
  Errors: 0

📋 Sample Logged Records (with JSON metadata extraction):

📄 Recent Change Log Records Preview (first 6 of 6 rows):
--------------------------------------------------------------------------------
 change_id  content_id                 file_name                                                 content_checksum  requires_faq_regeneration  page                                                         title domain  existing_faq_count          detected_at
        24        1006 Employee_Leave_Policy.pdf 1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca6b9abe19                          0     4 Employee Leave Policy - Bereavement and Leave Request Process     HR                   0 2025-10-20T03:28:10Z
        23    

## Step 7: Update FAQ Mappings (Checksum-Based Invalidation)

**Process FAQ updates using FAQUpdater**

This step updates the `faq_content_map` table based on detection results:

**Checksum-Based Logic**:
- **For pages requiring regeneration**: Invalidate all FAQs linked to that checksum
- **For preserved pages**: Update metadata (current location) if needed

**Benefits**:
- Single FAQ invalidation by checksum (handles all FAQs for that content)
- Metadata updates preserve FAQ validity while updating location
- No complex position tracking needed

In [15]:
print("\n" + "="*100)
print("STEP 3: UPDATE FAQ MAPPINGS")
print("="*100)

start_time = datetime.now()
log_info(f"Starting FAQ updates at {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

try:
    # Update FAQ mappings using FAQUpdater (checksum-based invalidation)
    # No adapter needed - direct processing of detection result
    faq_results = faq_updater.process_detection_results(
        faq_regen_result,
        detection_run_id=f"run_{start_time.strftime('%Y%m%d_%H%M%S')}"
    )
    
    faq_time = (datetime.now() - start_time).total_seconds()
    
    print(f"\n📊 FAQ Update Results (completed in {faq_time:.2f}s):")
    print(f"  FAQs Invalidated: {faq_results['faqs_invalidated']}")
    print(f"  FAQs Preserved: {faq_results['faqs_preserved']}")
    print(f"  FAQs Metadata Updated: {faq_results['faqs_metadata_updated']}")
    print(f"  Errors: {len(faq_results['errors'])}")
    
    if faq_results['errors']:
        log_warning(f"Encountered {len(faq_results['errors'])} errors during FAQ updates")
        for error in faq_results['errors'][:5]:
            print(f"    - {error}")
    
    # Show invalidated FAQs (if any)
    if faq_results['faqs_invalidated'] > 0:
        print("\n❌ Invalidated FAQs (require regeneration):")
        with sqlite3.connect(DB_PATH) as conn:
            invalidated_df = pd.read_sql_query("""
                SELECT 
                    map_id,
                    question_id,
                    answer_id,
                    content_checksum,
                    current_file_name,
                    json_extract(current_metadata, '$.current_page') as page,
                    invalidation_reason,
                    invalidated_by_change_id
                FROM faq_content_map
                WHERE is_valid = 0 
                  AND invalidated_by_change_id IS NOT NULL
                ORDER BY valid_until DESC
                LIMIT 20
            """, conn)
            
            preview_dataframe(invalidated_df, "Recently Invalidated FAQs", max_rows=20)
    
    # Show preserved FAQs
    if faq_results['faqs_preserved'] > 0:
        print(f"\n✅ {faq_results['faqs_preserved']} FAQs preserved (no regeneration needed)")
    
    log_success(f"FAQ mappings updated")
    
except Exception as e:
    log_error(f"FAQ update failed: {e}")
    raise

print("\n" + "="*100)


STEP 3: UPDATE FAQ MAPPINGS
ℹ️  Starting FAQ updates at 2025-10-19 23:28:10

📊 FAQ Update Results (completed in 0.00s):
  FAQs Invalidated: 0
  FAQs Preserved: 0
  FAQs Metadata Updated: 0
  Errors: 0
✅ FAQ mappings updated



## Step 8: Verification Queries (Simplified Schema)

Verify the detection, logging, and FAQ update results using the simplified schema.

In [16]:
print("\n" + "="*100)
print("VERIFICATION QUERIES (Simplified Schema)")
print("="*100)

with sqlite3.connect(DB_PATH) as conn:
    # 1. FAQ Regeneration Summary (from telemetry - already displayed above)
    print("1️⃣ FAQ Regeneration Summary (from detection telemetry):")
    print(f"  Pages requiring regeneration: {telemetry['pages_requiring_regeneration']}")
    print(f"  Pages not requiring regeneration: {telemetry['pages_not_requiring_regeneration']}")
    print(f"  FAQs to invalidate: {telemetry['faqs_to_invalidate']}")
    print(f"  FAQs preserved: {telemetry['faqs_preserved']}")
    
    # 2. Content Change Log Summary (simplified schema - no change_type)
    print("\n2️⃣ Content Change Log Summary (Simplified Schema with JSON metadata):")
    change_summary_df = pd.read_sql_query("""
        SELECT 
            requires_faq_regeneration,
            COUNT(*) as total_records,
            COUNT(DISTINCT file_name) as files_affected,
            COUNT(DISTINCT content_checksum) as unique_checksums,
            SUM(existing_faq_count) as total_faqs_affected,
            MIN(source_modified_at) as earliest_change,
            MAX(source_modified_at) as latest_change
        FROM content_change_log
        GROUP BY requires_faq_regeneration
        ORDER BY requires_faq_regeneration DESC
    """, conn)
    preview_dataframe(change_summary_df, "Change Log Summary by Regeneration Flag", max_rows=10)
    
    # 3. JSON Metadata Extraction Examples
    print("\n3️⃣ JSON Metadata Extraction Examples:")
    metadata_df = pd.read_sql_query("""
        SELECT 
            change_id,
            file_name,
            content_checksum,
            requires_faq_regeneration,
            json_extract(metadata, '$.page') as page,
            json_extract(metadata, '$.title_nme') as title,
            json_extract(metadata, '$.domain') as domain,
            json_extract(metadata, '$.service') as service,
            existing_faq_count
        FROM content_change_log
        ORDER BY change_id DESC
        LIMIT 10
    """, conn)
    preview_dataframe(metadata_df, "Metadata Extraction from JSON", max_rows=10)
    
    # 4. FAQ Content Map Status
    print("\n4️⃣ FAQ Content Map Status:")
    faq_map_summary = pd.read_sql_query("""
        SELECT 
            is_valid,
            COUNT(*) as total_mappings,
            COUNT(DISTINCT content_checksum) as unique_checksums,
            COUNT(DISTINCT current_file_name) as files,
            COUNT(DISTINCT question_id) as unique_questions
        FROM faq_content_map
        GROUP BY is_valid
        ORDER BY is_valid DESC
    """, conn)
    preview_dataframe(faq_map_summary, "FAQ Map Summary by Validity", max_rows=10)
    
    # 5. Content Repository Status
    print("\n5️⃣ Content Repository Status:")
    content_stats = pd.read_sql_query("""
        SELECT 
            COUNT(*) as total_pages,
            COUNT(DISTINCT raw_file_nme) as unique_files,
            COUNT(DISTINCT content_checksum) as unique_checksums,
            MAX(last_modified_dt) as latest_modification
        FROM content_repo
        WHERE file_status = 'Active'
    """, conn)
    preview_dataframe(content_stats, "Content Repository Status", max_rows=10)

print("\n" + "="*100)
log_success("Verification queries completed")


VERIFICATION QUERIES (Simplified Schema)
1️⃣ FAQ Regeneration Summary (from detection telemetry):
  Pages requiring regeneration: 2
  Pages not requiring regeneration: 1
  FAQs to invalidate: 0
  FAQs preserved: 0

2️⃣ Content Change Log Summary (Simplified Schema with JSON metadata):

📄 Change Log Summary by Regeneration Flag Preview (first 2 of 2 rows):
--------------------------------------------------------------------------------
 requires_faq_regeneration  total_records  files_affected  unique_checksums  total_faqs_affected          earliest_change            latest_change
                         1              4               1                 2                    0 2025-10-17T17:01:58.000Z 2025-10-17T17:01:58.000Z
                         0              2               1                 1                    0 2025-10-17T17:01:58.000Z 2025-10-17T17:01:58.000Z
--------------------------------------------------------------------------------

3️⃣ JSON Metadata Extraction Examples

## Appendix: Advanced JSON Metadata Queries

Examples of querying JSON metadata fields in the simplified schema.

In [17]:
print("\n" + "="*100)
print("✅ UPDATE DETECTION COMPLETE (Simplified Checksum-Centric Architecture)")
print("="*100)

with sqlite3.connect(DB_PATH) as conn:
    # Final statistics (simplified schema - no content_diffs table)
    final_stats = pd.read_sql_query("""
        SELECT 
            'content_repo' as table_name,
            COUNT(*) as record_count
        FROM content_repo
        UNION ALL
        SELECT 'content_change_log', COUNT(*) FROM content_change_log
        UNION ALL
        SELECT 'faq_content_map', COUNT(*) FROM faq_content_map
        UNION ALL
        SELECT 'faq_questions', COUNT(*) FROM faq_questions
        UNION ALL
        SELECT 'faq_answers', COUNT(*) FROM faq_answers
    """, conn)
    
    print("\n📊 Final Database Statistics:")
    for _, row in final_stats.iterrows():
        print(f"  {row['table_name']}: {row['record_count']} records")
    
    # Processing summary (using telemetry + results)
    print("\n📋 Processing Summary (Checksum-Centric Architecture):")
    print(f"  Detection Period: Since {SINCE_DATE}")
    print(f"  Pages Requiring Regeneration: {telemetry['pages_requiring_regeneration']}")
    print(f"  Pages Preserving FAQs: {telemetry['pages_not_requiring_regeneration']}")
    print(f"  FAQs Invalidated: {faq_results.get('faqs_invalidated', 0)}")
    print(f"  FAQs Preserved: {faq_results.get('faqs_preserved', 0)}")
    print(f"  Detection Duration: {telemetry['detection_duration_seconds']:.3f}s")
    
    # Database size
    import os
    if DB_PATH.exists():
        db_size = os.path.getsize(DB_PATH)
        print(f"\n💾 Database Size: {db_size:,} bytes ({db_size/1024/1024:.2f} MB)")
    
    # Telemetry summary
    print("\n📡 Telemetry Summary:")
    print(f"  Files Processed: {telemetry['files_processed']}")
    print(f"  Total Pages Analyzed: {telemetry['total_pages_analyzed']}")
    print(f"  Baseline Checksums Loaded: {telemetry['total_baseline_checksums_loaded']}")
    print(f"  Warnings: {len(telemetry['warnings'])}")
    
    # Architecture notes
    print("\n🏗️  Architecture Notes (Simplified Checksum-Centric Model):")
    print("  ✅ No adapter needed - direct detection result processing")
    print("  ✅ Binary decision model - requires_faq_regeneration (0/1)")
    print("  ✅ JSON metadata storage - flexible optional fields")
    print("  ✅ Checksum-based FAQ invalidation - single operation per content")
    print("  ✅ No content_diffs table - simpler schema")
    print("  ✅ Built-in telemetry - comprehensive metrics included")
    
    # Recommendations
    print("\n💡 Next Steps:")
    if telemetry['pages_requiring_regeneration'] > 0:
        print(f"  - Regenerate FAQs for {telemetry['pages_requiring_regeneration']} pages with new/modified content")
        print(f"  - {telemetry['faqs_preserved']} existing FAQs can be reused (no regeneration needed)")
        print(f"  - Use v_regeneration_queue VIEW to prioritize FAQ generation")
    else:
        print("  - No FAQ regeneration needed! All content checksums exist in baseline.")
    
    if telemetry['warnings']:
        print(f"  - Review {len(telemetry['warnings'])} warnings from detection")
    
    # Show regeneration queue (if any)
    if telemetry['pages_requiring_regeneration'] > 0:
        print("\n📝 Regeneration Queue (prioritized by FAQ count + recency):")
        regen_queue = pd.read_sql_query("""
            SELECT * FROM v_regeneration_queue
            LIMIT 10
        """, conn)
        preview_dataframe(regen_queue, "Top 10 Pages for FAQ Regeneration", max_rows=10)

print("\n" + "="*100)
log_success("FAQ regeneration detection workflow complete!")


✅ UPDATE DETECTION COMPLETE (Simplified Checksum-Centric Architecture)

📊 Final Database Statistics:
  content_repo: 6 records
  content_change_log: 6 records
  faq_content_map: 0 records
  faq_questions: 10 records
  faq_answers: 10 records

📋 Processing Summary (Checksum-Centric Architecture):
  Detection Period: Since 2025-10-17T17:01:51.100Z
  Pages Requiring Regeneration: 2
  Pages Preserving FAQs: 1
  FAQs Invalidated: 0
  FAQs Preserved: 0
  Detection Duration: 0.011s

💾 Database Size: 245,760 bytes (0.23 MB)

📡 Telemetry Summary:
  Files Processed: 1
  Total Pages Analyzed: 3
  Baseline Checksums Loaded: 3
  Warnings: 0

🏗️  Architecture Notes (Simplified Checksum-Centric Model):
  ✅ No adapter needed - direct detection result processing
  ✅ Binary decision model - requires_faq_regeneration (0/1)
  ✅ JSON metadata storage - flexible optional fields
  ✅ Checksum-based FAQ invalidation - single operation per content
  ✅ No content_diffs table - simpler schema
  ✅ Built-in teleme

In [21]:
with sqlite3.connect(DB_PATH) as conn:
    # Query all records from content_repo
    all_records_df = pd.read_sql_query("""
        SELECT *
        FROM content_repo
        ORDER BY ud_source_file_id
    """, conn)

all_records_df

ud_source_file_id domain            service  orgoid associateoid  \
0               1001     HR  Policy Management  ORG001     ASSOC001   
1               1002     HR  Policy Management  ORG001     ASSOC001   
2               1003     HR  Policy Management  ORG001     ASSOC001   
3               1004     HR  Policy Management  ORG001     ASSOC001   
4               1005     HR  Policy Management  ORG001     ASSOC001   
5               1006     HR  Policy Management  ORG001     ASSOC001   

                raw_file_nme raw_file_type  raw_file_version_nbr  \
0  Employee_Leave_Policy.pdf          .pdf                     1   
1  Employee_Leave_Policy.pdf          .pdf                     1   
2  Employee_Leave_Policy.pdf          .pdf                     1   
3  Employee_Leave_Policy.pdf          .pdf                     2   
4  Employee_Leave_Policy.pdf          .pdf                     2   
5  Employee_Leave_Policy.pdf          .pdf                     1   

   raw_file_page_nbr                               source_url_txt  \
0                  1                                         None   
1                  2                                         None   
2                  3                                         None   
3                  2  https://intranet.example.com/policies/leave   
4                  3  https://intranet.example.com/policies/leave   
5                  4  https://intranet.example.com/policies/leave   

  parent_location_txt                    raw_file_path  \
0                None  /test/Employee_Leave_Policy.pdf   
1                None  /test/Employee_Leave_Policy.pdf   
2                None  /test/Employee_Leave_Policy.pdf   
3           /policies  /test/Employee_Leave_Policy.pdf   
4           /policies  /test/Employee_Leave_Policy.pdf   
5           /policies  /test/Employee_Leave_Policy.pdf   

                                  extracted_markdown_file_path  \
0  c:\Users\praka\projects\FAQ\FAQ_update\docs\test_sample_...   
1  c:\Users\praka\projects\FAQ\FAQ_update\docs\test_sample_...   
2  c:\Users\praka\projects\FAQ\FAQ_update\docs\test_sample_...   
3  c:\Users\praka\projects\FAQ\FAQ_update\test_data_version...   
4  c:\Users\praka\projects\FAQ\FAQ_update\test_data_version...   
5  c:\Users\praka\projects\FAQ\FAQ_update\test_data_version...   

  extracted_layout_file_path  \
0              layout_p1.txt   
1              layout_p2.txt   
2              layout_p3.txt   
3           layout_p2_v2.txt   
4          layout_p3_new.txt   
5              layout_p3.txt   

                                                     title_nme  \
0                                        Employee Leave Policy   
1                                        Employee Leave Policy   
2                                        Employee Leave Policy   
3                           Employee Leave Policy - Sick Leave   
4    Employee Leave Policy - Remote Work During Parental Leave   
5  Employee Leave Policy - Bereavement and Leave Request Pr...   

                     breadcrumb_txt                   content_tags_txt  \
0                              None                               None   
1                              None                               None   
2                              None                               None   
3  Policies > HR > Leave Management         leave;sick;medical;absence   
4  Policies > HR > Leave Management     leave;parental;remote;flexible   
5  Policies > HR > Leave Management  leave;bereavement;process;request   

   version_nbr                                             content_checksum  \
0            1  620c75e45bb1c3a1225a411c9821805c866c60fe8e6a61c8586ab3ac...   
1            1  b04ed99ce1bf45e408ed7a519c4d4d65554fcccaa7e3fbab8ba37639...   
2            1  1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca...   
3            2  ce9d64d784d0d2618919b6d3a49601785095b21f968c06f71b92a8e1...   
4            2  4aefd3fb027461ca50f27186787cc28e0d1a90c91ed4764561f509ca

In [19]:
with sqlite3.connect(DB_PATH) as conn:
    # Query all records from content_repo
    all_content_change_df = pd.read_sql_query("""
        SELECT *
        FROM content_change_log
    """, conn)

all_content_change_df

change_id  content_id  \
0         19        1004   
1         20        1005   
2         21        1006   
3         22        1004   
4         23        1005   
5         24        1006   

                                              content_checksum  \
0  ce9d64d784d0d2618919b6d3a49601785095b21f968c06f71b92a8e1...   
1  4aefd3fb027461ca50f27186787cc28e0d1a90c91ed4764561f509ca...   
2  1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca...   
3  ce9d64d784d0d2618919b6d3a49601785095b21f968c06f71b92a8e1...   
4  4aefd3fb027461ca50f27186787cc28e0d1a90c91ed4764561f509ca...   
5  1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca...   

                   file_name  requires_faq_regeneration  \
0  Employee_Leave_Policy.pdf                          1   
1  Employee_Leave_Policy.pdf                          1   
2  Employee_Leave_Policy.pdf                          0   
3  Employee_Leave_Policy.pdf                          1   
4  Employee_Leave_Policy.pdf                          1   
5  Employee_Leave_Policy.pdf                          0   

                                                      metadata  \
0  {"page": 2, "domain": "HR", "service": "Policy Managemen...   
1  {"page": 3, "domain": "HR", "service": "Policy Managemen...   
2  {"page": 4, "domain": "HR", "service": "Policy Managemen...   
3  {"page": 2, "domain": "HR", "service": "Policy Managemen...   
4  {"page": 3, "domain": "HR", "service": "Policy Managemen...   
5  {"page": 4, "domain": "HR", "service": "Policy Managemen...   

            detected_at        source_modified_at  existing_faq_count  \
0  2025-10-20T03:26:57Z  2025-10-17T17:01:58.000Z                   0   
1  2025-10-20T03:26:57Z  2025-10-17T17:01:58.000Z                   0   
2  2025-10-20T03:26:57Z  2025-10-17T17:01:58.000Z                   0   
3  2025-10-20T03:28:10Z  2025-10-17T17:01:58.000Z                   0   
4  2025-10-20T03:28:10Z  2025-10-17T17:01:58.000Z                   0   
5  2025-10-20T03:28:10Z  2025-10-17T17:01:58.000Z                   0   

      detection_run_id                since_date  
0  run_20251019_232657  2025-10-17T17:01:51.100Z  
1  run_20251019_232657  2025-10-17T17:01:51.100Z  
2  run_20251019_232657  2025-10-17T17:01:51.100Z  
3  run_20251019_232810  2025-10-17T17:01:51.100Z  
4  run_20251019_232810  2025-10-17T17:01:51.100Z  
5  run_20251019_232810  2025-10-17T17:01:51.100Z

In [20]:
with sqlite3.connect(DB_PATH) as conn:
    # Query all records from content_repo
    all_faq_map_df = pd.read_sql_query("""
        SELECT *
        FROM faq_content_map
    """, conn)

all_faq_map_df

Empty DataFrame
Columns: [map_id, question_id, answer_id, content_checksum, current_content_id, current_file_name, original_content_id, created_for_version, current_metadata, original_metadata, is_valid, valid_from, valid_until, invalidation_reason, invalidated_by_change_id, replaced_by_map_id, confidence_score, generation_method, created_at, last_validated_at]
Index: []